In [1]:
%matplotlib inline
import math as m; import numpy as np; import matplotlib.pyplot as plt; import matplotlib.colors
import pandas as pd; import os; import pathlib; import string; import datetime; import h5py

directory = 'D:\\Jupiter_satellites_data\\'
directory = 'C:\\Users\\jacob\\Documents\\Boston University\\Astronomy PhD\\Research\\Wen Li Group\\csp-research\\juno_data\\'

In [3]:
def find_coords(juno_t_array, moon_indices):
    d_eq_list = np.array([]); MS_list = np.array([]); MLAT_list = np.array([]); MLT_list = np.array([]); 
    for i, juno_t_str in zip(moon_indices, juno_t_array):
        # set moon labels, criteria, radii, and years
        sat_list = ['Io', 'Europa', 'Ganymede']
        d_eq_VAR = [75, 75, 75]
        RI = 0.0261 # Io radius in RJ
        RE = 0.0223 # Europa radius in RJ
        RG = 0.0377 # Ganymede radius in RJ
        R_list = np.array([RI, RE, RG]); R_labels = np.array(['RI', 'RE', 'RG'])
        years = np.arange(2016, 2022+1)
    
        # read in M-Shell data from Xiaochen code
        XC_file = 'jupiterMoonsJunoMShell_ALL_v1.0.txt'
        XC_fields = ['Year', 'Month', 'Date', 'Hour', 'Minute', 'Juno_MSHELL', 'Io_MSHELL', 'Europa_MSHELL', 'Ganymede_MSHELL']
        XC_data = pd.read_csv(directory + XC_file, names = XC_fields, usecols = XC_fields, sep = '\s+', header = 5)
        yyyy_XC = XC_data['Year'].values; mm_XC = XC_data['Month'].values; dd_XC = XC_data['Date'].values;
        hh_XC = XC_data['Hour'].values; mn_XC = XC_data['Minute'].values; MSHELL_Juno = XC_data['Juno_MSHELL'].values;
        MSHELL_Io = XC_data['Io_MSHELL'].values; MSHELL_Europa = XC_data['Europa_MSHELL'].values;
        MSHELL_Ganymede = XC_data['Ganymede_MSHELL'].values
    
        # read in MLT and MLAT data from Xiaochen data file
        yyyy_pos = np.array([]); mm_pos = np.array([]); dd_pos = np.array([]); hh_pos = np.array([]); mn_pos = np.array([]);
        MLAT_Juno = np.array([]); MLT_Juno = np.array([]); MLAT_Io = np.array([]); MLT_Io = np.array([]);
        MLAT_Europa = np.array([]); MLT_Europa = np.array([]); MLAT_Ganymede = np.array([]); MLT_Ganymede = np.array([]);
        for year in years:
            pos_file = f'jupiterMoonsMLTMLATInfo_{year}_v1.0.txt'
            pos_fields = ['Year', 'Month', 'Date', 'Hour', 'Minute', 'MLAT_Juno [degree]', 'MLT_Juno [hr]','MLAT_Io [degree]', \
                      'MLT_Io [hr]','MLAT_Europa [degree]', 'MLT_Europa [hr]', 'MLAT_Ganymede [degree]', 'MLT_Ganymede [hr]']
            pos_data = pd.read_csv(directory + pos_file, names = pos_fields, usecols = pos_fields, sep = '\s+', header = 10)
            # add to full list
            yyyy_pos_i = pos_data['Year'].values; mm_pos_i = pos_data['Month'].values;
            dd_pos_i = pos_data['Date'].values; hh_pos_i = pos_data['Hour'].values; mn_pos_i = pos_data['Minute'].values;
            MLAT_Juno_i = pos_data['MLAT_Juno [degree]'].values; MLT_Juno_i = pos_data['MLT_Juno [hr]'].values;
            MLAT_Io_i = pos_data['MLAT_Io [degree]'].values; MLT_Io_i = pos_data['MLT_Io [hr]'].values;
            MLAT_Europa_i = pos_data['MLAT_Europa [degree]'].values; MLT_Europa_i = pos_data['MLT_Europa [hr]'].values;
            MLAT_Ganymede_i = pos_data['MLAT_Ganymede [degree]'].values; MLT_Ganymede_i = pos_data['MLT_Ganymede [hr]'].values;
            yyyy_pos = np.append(yyyy_pos, yyyy_pos_i); mm_pos = np.append(mm_pos, mm_pos_i);
            dd_pos = np.append(dd_pos, dd_pos_i); hh_pos = np.append(hh_pos, hh_pos_i); mn_pos = np.append(mn_pos, mn_pos_i);
            MLAT_Juno = np.append(MLAT_Juno, MLAT_Juno_i); MLT_Juno = np.append(MLT_Juno, MLT_Juno_i); 
            MLAT_Io = np.append(MLAT_Io, MLAT_Io_i); MLT_Io = np.append(MLT_Io, MLT_Io_i);
            MLAT_Europa = np.append(MLAT_Europa, MLAT_Europa_i); MLT_Europa = np.append(MLT_Europa, MLT_Europa_i);
            MLAT_Ganymede = np.append(MLAT_Ganymede, MLAT_Ganymede_i); MLT_Ganymede = np.append(MLT_Ganymede, MLT_Ganymede_i);

        # define time index
        round_mn = round(int(juno_t_str[14:16])/30)*30
        round_hh = int(juno_t_str[11:13])
        if round_mn > 45:
            round_mn = 0
            round_hh = int(juno_t_str[11:13]) + 1
            if round_hh >= 24:
                print('find_coords function has encountered an error: minute rounding break')
        t_index_XC = (yyyy_XC == int(juno_t_str[0:4])) & (mm_XC == int(juno_t_str[5:7])) & (dd_XC == int(juno_t_str[8:10])) & (hh_XC == round_hh) & (mn_XC == round_mn)
        t_index_pos = (yyyy_pos == int(juno_t_str[0:4])) & (mm_pos == int(juno_t_str[5:7])) & (dd_pos == int(juno_t_str[8:10])) & (hh_pos == int(juno_t_str[11:13])) & (mn_pos == int(juno_t_str[14:16]))
        # find d_eq, M-shell, MLT, and MLAT for Juno
        MSHELL_Sat = np.array([MSHELL_Io[t_index_XC], MSHELL_Europa[t_index_XC], MSHELL_Ganymede[t_index_XC]])
        MLT_Sat = np.array([MLT_Io[t_index_pos], MLT_Europa[t_index_pos], MLT_Ganymede[t_index_pos]])
        x_Sat = MSHELL_Sat[i]*np.cos(MLT_Sat[i]*(np.pi/12))/R_list[i]
        y_Sat = MSHELL_Sat[i]*np.sin(MLT_Sat[i]*(np.pi/12))/R_list[i]
        x_Juno = MSHELL_Juno[t_index_XC]*np.cos(MLT_Juno[t_index_pos]*(np.pi/12))/R_list[i]
        y_Juno = MSHELL_Juno[t_index_XC]*np.sin(MLT_Juno[t_index_pos]*(np.pi/12))/R_list[i]

        d_eq = np.sqrt((x_Juno - x_Sat)**2 + (y_Juno - y_Sat)**2)[0]; MS = MSHELL_Juno[t_index_XC][0]; MLAT = MLAT_Juno[t_index_pos][0]; MLT = MLT_Juno[t_index_pos][0]
        d_eq_list = np.append(d_eq_list, d_eq); MS_list = np.append(MS_list, MS); MLAT_list = np.append(MLAT_list, MLAT); MLT_list = np.append(MLT_list, MLT);
    return d_eq_list, MS_list, MLAT_list, MLT_list

In [3]:
juno_t_array = np.array(['2022-04-09/12:15'])
moon_indices = np.array([0])
d_eq_list, MS_list, MLAT_list, MLT_list = find_coords(juno_t_array, moon_indices)

In [4]:
print(f'{d_eq_list[0]}, {MS_list[0]}, {MLAT_list[0]}, {MLT_list[0]}')

26.352365253440716, 6.6097565000000005, 16.63, 19.65
